<a href="https://colab.research.google.com/github/tbaeumel/transformer-digit-arithmetic/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get the data from the repo -> anonymize!

In [ ]:
!git clone https://github.com/tbaeumel/transformer-digit-arithmetic.git

In [ ]:
%cd transformer-digit-arithmetic

Get pyvene
Olmo2 was not in pyvene - i wrote an extension so that olmo2 can be an intervenable model.
 has to be anonymized

In [ ]:
# pip uninstall -y pyvene  # pyvene has to be uninstalled if a standard version is currently installed
# pip install pyvene
!pip install git+https://github.com/tbaeumel/pyvene

In [ ]:
import pyvene as pv
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import sys
import torch.nn.functional as F
from pyvene import embed_to_distrib, top_vals, format_token
from pyvene import (
    ZeroIntervention,
    IntervenableModel,
    VanillaIntervention, Intervention,
    RepresentationConfig,
    IntervenableConfig,
    ConstantSourceIntervention,
    LocalistRepresentationIntervention
)
from huggingface_hub import login
import re
import numpy
from tqdm import tqdm
from matplotlib import pyplot as plt
import transformers
import os

## Load model

Llama 70B wont work on colab

llama 8b and olmo 2 7b will work on l4 or a100

people have to login with their huggingface key to get access to the llama models

In [ ]:
# Login to Huggingface to get access to model parameters
# Paste your token here
login('')

In [ ]:
# choose your model
model_name = "Llama-3-8B" # Llama-3-70B # Olmo-2-7B

In [ ]:
models = {"Llama-3-8B": "meta-llama/Meta-Llama-3-8B", "Llama-3-70B": "meta-llama/Meta-Llama-3-70B", "Olmo-2-7B": "allenai/OLMo-2-1124-7B" }

# Set parameters
params = {
    'model_name': models[model_name],
    'device': "cuda" if torch.cuda.is_available() else "cpu"
}

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(params['model_name'])
model = AutoModelForCausalLM.from_pretrained(params['model_name'], torch_dtype=torch.float16).to(params['device'])

# Confirm device
print(f"Using device: {params['device']}")

device = "cuda:0" if torch.cuda.is_available() else "cpu"
if torch.cuda.is_available():
  print("Using GPU")

# Digit circuit Intervention

Choose:
- thresholds
- digit position
- task (operator)
- operand (which intervention dataset)

this can be changed to standard values for the thresholds and layers (see paper Section X), so that based on model and digti position the best threshold is chosen


In [ ]:
# labels
label = "hundredth" # "tenth" # "unit"
# task (operator)
task = "addition" # "subtraction"
# which oeprand
operand = "op1" # "op2"

In [ ]:
# Load the JSON file
input_file = f"Intervention_Data/intervention_data_{task}_{operand}.json"
with open(input_file, "r") as f:
    data = json.load(f)

all of this should be stored in some sort of data look up file

In [ ]:
# layer sets to intervene on based on model, task, and operand
# TODO do this elegantly

key = f"{model_name}_{task}_{operand}"

layer_sets = {
    "Llama-3-8B_addition_op1": [16, 17, 18, 19, 20, 21, 22, 23, 24],
    "Llama-3-8B_addition_op2": [15, 16, 17, 18, 19, 20, 21, 22, 23, 24],
    "Llama-3-8B_subtraction_op1": [16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28],
    "Llama-3-8B_subtraction_op2": [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28],
    "Llama-3-70B_addition_op1": [39, ..., 56],
    "Llama-3-70B_addition_op2": [39, ..., 56],
    "Llama-3-70B_subtraction_op1": [39, ..., 58],
    "Llama-3-70B_subtraction_op2": [39, ..., 58],
    "Olmo-2-7B_addition_op1": [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
    "Olmo-2-7B_addition_op2": [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30],
    "Olmo-2-7B_subtraction_op1": [19, 20, 21, 22, 23, 24, 25, 26, 27],
    "Olmo-2-7B_subtraction_op2": [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27],
    }

layer_set = layer_sets[key]

# TODO add llama 70 when results are there (best threshold)
key = key+"_"+label
threshold_map = {
    "Llama-3-8B_addition_op1_hundredth": 0.8,
    "Llama-3-8B_addition_op1_tenth": 0.4,
    "Llama-3-8B_addition_op1_unit": 0.6,
    "Llama-3-8B_addition_op2_hundredth": 0.9,
    "Llama-3-8B_addition_op2_tenth": 0.5,
    "Llama-3-8B_addition_op2_unit": 0.6,
    "Llama-3-8B_subtraction_op1_hundredth": 0.8,
    "Llama-3-8B_subtraction_op1_tenth": 0.5,
    "Llama-3-8B_subtraction_op1_unit": 0.6,
    "Llama-3-8B_subtraction_op2_hundredth": 0.9,
    "Llama-3-8B_subtraction_op2_tenth": 0.5,
    "Llama-3-8B_subtraction_op2_unit": 0.6,
    "Olmo-2-7B_addition_op1_hundredth": 0.8,
    "Olmo-2-7B_addition_op1_tenth": 0.8,
    "Olmo-2-7B_addition_op1_unit": 0.4,
    "Olmo-2-7B_addition_op2_hundredth": 0.8,
    "Olmo-2-7B_addition_op2_tenth": 0.8,
    "Olmo-2-7B_addition_op2_unit": 0.4,
    "Olmo-2-7B_subtraction_op1_hundredth<": 0.8,
    "Olmo-2-7B_subtraction_op1_tenth": 0.8,
    "Olmo-2-7B_subtraction_op1_unit": 0.5,
    "Olmo-2-7B_subtraction_op2_hundredth": 0.9,
    "Olmo-2-7B_subtraction_op2_tenth": 0.9,
    "Olmo-2-7B_subtraction_op2_unit": 0.5,
}

threshold = threshold_map[key]

print(layer_set)
print(threshold)

people should have the option to overwrite layer set and threshold by themselves so that they can experiment

Extract the MLP dimensions that have a fisher score above the chosen threshold

In [ ]:
# According to the threshold chosen, get the MLP dimensions per layer to be intervened on, i.e., the MLP neurons that belong to the digit circuit at that threshold

# Open the right fisher score file
with open(f"Fisher_Scores/{model_name}/{task}/fisher_scores_{label}.json", "r") as file:
    fisher_scores_data = json.load(file)

# Output dictionary
layer_subspaces_map = {}

# Iterate through selected layers
for layer in layer_set:
    key = f"layer_{layer}"
    if key in fisher_scores_data:
        values = fisher_scores_data[key]
        indices_above_threshold = [i for i, val in enumerate(values) if val > threshold]
        layer_subspaces_map[layer] = indices_above_threshold

# Result
print(layer_subspaces_map)

In [ ]:
################################################
# Perform Digit-Circuit Interventions on each data point #
################################################

# Iterate through each query in the dataset
for j, entry in tqdm(enumerate(data)):
    data_entry = []
    model_layers = model.config.num_hidden_layers
    window_size = 1

    sentence = entry["one_shot_base"]
    sentence_intervention = entry["one_shot_source"]

    base = tokenizer(sentence, return_tensors="pt").to(device)

    # Number of tokens
    tokenized_input = tokenizer(sentence, return_tensors="pt", return_offsets_mapping=True)
    input_ids = tokenized_input["input_ids"].to(device)
    num_tokens = input_ids.shape[1]

    ############################
    # Clean Run for comparison #
    ############################

    inputs = [tokenizer(sentence, return_tensors="pt").to(device),]
    res = model(**inputs[0])

    distrib = res.logits
    logits = distrib[0][-1]

    # Apply softmax to get probabilities
    probabilities = F.softmax(logits, dim=-1)

    # Get the top 10 tokens and their probabilities
    top_k = 50
    top_k_probs, top_k_indices = torch.topk(probabilities, top_k)

    # Convert indices to tokens
    top_k_tokens = [tokenizer.decode(index.item()) for index in top_k_indices]

    # Collect the data
    data_temp = []
    for token, prob in zip(top_k_tokens, top_k_probs):
        data_temp.append({
            "token": token,
            "prob": prob.detach().cpu().item()
        })

    data_entry.append({"run": "clean", "top_100": data_temp})

    ###############################################
    # Interchange Interventions across layer sets #
    ###############################################

    # Get the index of the last token using len()
    last_token_index = len(base['input_ids'][0]) - 1  # Use len() to get the length of the sequence

    # Create intervention for specific layers
    config = pv.IntervenableConfig([{
        "layer": l,
        "component": "mlp_output",
        "intervention_type": VanillaIntervention
        } for l in layer_set] # Pass a list instead of a single layer
    )

    pv_model = pv.IntervenableModel(config, model=model)

    # Define list of subspaces based on the layer_subspaces_map
    # Create an empty list to store the corresponding subspaces for each layer in layer_set
    subspaces = []

    # Loop over the layers in the current layer_set and fetch corresponding subspaces
    for layer in layer_set:
        subspaces.append(layer_subspaces_map[layer])

    # run an interchange intervention
    _, intervened_outputs = pv_model(
      # the base input
      base=tokenizer(sentence, return_tensors = "pt").to(device),
      # the source input
      sources=tokenizer(sentence_intervention, return_tensors = "pt").to(device),
      # the location to intervene at (last token)
      unit_locations={"sources->base": last_token_index},
      subspaces = subspaces
    )

    distrib = intervened_outputs.logits
    logits = distrib[0][-1]

    # Apply softmax to get probabilities
    probabilities = F.softmax(logits, dim=-1)

    # Get the top 10 tokens and their probabilities
    top_k = 100
    top_k_probs, top_k_indices = torch.topk(probabilities, top_k)

    # Convert indices to tokens
    top_k_tokens = [tokenizer.decode(index.item()) for index in top_k_indices]

    # Collect the data
    data_temp = []
    for token, prob in zip(top_k_tokens, top_k_probs):
        data_temp.append({
            "token": token,
            "prob": prob.detach().cpu().item()
        })

    data_entry.append({"run": "intervened", "top_100": data_temp})

    df = pd.DataFrame(data_entry)

    output_dir = f"Interventions/"
    os.makedirs(output_dir, exist_ok=True)  # Create the folder if it doesn't exist

    df.to_csv(f"{output_dir}/intervention_{model_name}_{task}_{operand}_{label}_threshold_{threshold}_{j}.csv")

# Evaluation

In [ ]:
import os
import json
import pandas as pd
import ast
import re

variant_labels = ["bbb", "bbs", "bsb", "bss", "sbb", "sbs", "ssb", "sss"]
folder_path = "Interventions/"

# Initialize accumulators: sums and counts for each run and variant label
accumulated_probs = {
    "clean": {label: 0.0 for label in variant_labels},
    "intervened": {label: 0.0 for label in variant_labels}
}
counts = {
    "clean": {label: 0 for label in variant_labels},
    "intervened": {label: 0 for label in variant_labels}
}

# Get sorted CSV files
csv_files = [f for f in os.listdir(folder_path) if re.match(r"intervention_.*_(\d+)\.csv", f)]
csv_files.sort(key=lambda x: int(re.search(r"_(\d+)", x).group(1)))

for file_name in csv_files:
    index_match = re.search(r"_(\d+)\.csv$", file_name)
    i = int(index_match.group(1)) if index_match else None
    if i is None:
        continue

    print(f"Processing example {i} from file {file_name}")
    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)

    # Map variant labels to target tokens from your loaded `data`
    variant_token_map = {k: str(v) for k, v in data[i]["result_variants"].items()}

    for _, row in df.iterrows():
        run_type = row["run"]  # "clean" or "intervened"
        try:
            token_probs = ast.literal_eval(row["top_100"])
        except Exception as e:
            print(f"Error parsing token probs in {file_name}, run={run_type}: {e}")
            continue

        for variant_label, target_token in variant_token_map.items():
            for entry in token_probs:
                if entry["token"] == target_token:
                    prob = entry["prob"]
                    accumulated_probs[run_type][variant_label] += prob
                    counts[run_type][variant_label] += 1

# Compute averages over all examples and all files
averages = {
    run_type: {
        label: (accumulated_probs[run_type][label] / counts[run_type][label]) if counts[run_type][label] > 0 else 0
        for label in variant_labels
    }
    for run_type in ["clean", "intervened"]
}

# Save averages to JSON
output_file = "average_probabilities.json"
with open(output_file, "w") as f:
    json.dump(averages, f, indent=4)

print("Averages computed and saved to", output_file)

# Visualization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming `averages` dict is computed as before:
# averages = {
#   "clean": {...},
#   "intervened": {...}
# }

variant_labels = list(averages["clean"].keys())

clean_vals = [averages["clean"][label] for label in variant_labels]
intervened_vals = [averages["intervened"][label] for label in variant_labels]
diff_vals = [i - c for i, c in zip(intervened_vals, clean_vals)]

x = np.arange(len(variant_labels))
width = 0.25

fig, ax = plt.subplots(figsize=(12, 6))

bars1 = ax.bar(x - width, clean_vals, width, label='Clean', color='tab:blue')
bars2 = ax.bar(x, intervened_vals, width, label='Intervened', color='tab:orange')
bars3 = ax.bar(x + width, diff_vals, width, label='Difference (Clean - Intervened)', color='tab:green')

ax.set_ylabel('Average Probability')
ax.set_title('Average Probabilities per Variant Label')
ax.set_xticks(x)
ax.set_xticklabels(variant_labels)
ax.legend()
ax.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## Flip Rate

In [ ]:
import os
import ast
import json
import pandas as pd
import os
import json
import pandas as pd
import ast
import re
import sys

# Flip rate calculation
expected_variant_map = {
    "unit": "bbs",
    "tenth": "bsb",
    "hundredth": "sbb"
}

# Use your current label to find the expected variant
expected_variant = expected_variant_map[label]  # 'label' must be defined in your loop or global scope

flip_count = 0
total_count = 0

for file_name in csv_files:
    index_match = re.search(r"_(\d+)\.csv$", file_name)
    i = int(index_match.group(1)) if index_match else None
    if i is None:
        continue

    file_path = os.path.join(folder_path, file_name)
    df = pd.read_csv(file_path)

    # Get token mapping for this datapoint
    variant_token_map = {k: str(v) for k, v in data[i]["result_variants"].items()}

    # Dictionary to store best variant per run
    best_variant = {}

    for _, row in df.iterrows():
        run_type = row["run"]
        try:
            token_probs = ast.literal_eval(row["top_100"])
        except Exception as e:
            print(f"Error parsing token probs in {file_name}, run={run_type}: {e}")
            continue

        # Find the best matching variant for this run
        highest_prob = -1
        predicted_variant = None

        for variant_label, token in variant_token_map.items():
            for entry in token_probs:
                if entry["token"] == token:
                    if entry["prob"] > highest_prob:
                        highest_prob = entry["prob"]
                        predicted_variant = variant_label

        best_variant[run_type] = predicted_variant

    # Count flip: clean was bbb, intervened is expected variant
    if best_variant.get("clean") == "bbb" and best_variant.get("intervened") == expected_variant:
        flip_count += 1

    total_count += 1

flip_rate = flip_count / total_count if total_count > 0 else 0
print(f"\n Flip Rate (from 'bbb' → '{expected_variant}') = {flip_rate:.3f} over {total_count} examples.")